# LightGBMにてモデル構築から提出までの骨組みを作る


## import

In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import copy
import joblib

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import  f1_score

plt.rcParams['font.family'] = 'MS Gothic'

# 行数と列数を設定
pd.set_option('display.max_rows', None)  # すべての行を表示
pd.set_option('display.max_columns', None)  # すべての列を表示
pd.set_option('display.max_colwidth', None)  # 各列の表示幅を制限しない

MODE = 'LOCAL'

## データの読み込み

In [26]:
MODE = 'LOCAL'

if MODE == 'LOCAL':
    train = pd.read_csv('../../0_data/train.csv')
    test = pd.read_csv("../../0_data/test.csv")
    train_demo = pd.read_csv('../../0_data/train_demographics.csv')
    test_demo = pd.read_csv("../../0_data/test_demographics.csv")

    EXPORT_DIR = Path("../4_result")

if MODE == 'KAGGLE':
    import kaggle_evaluation.cmi_inference_server

    train = pd.read_csv("/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv")
    test = pd.read_csv("/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv")
    train_demo = pd.read_csv('/kaggle/input/cmi-detect-behavior-with-sensor-data/train_demographics.csv')
    test_demo = pd.read_csv("/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv")

    EXPORT_DIR = Path("./")

## データの前処理
LightGBMはNULLもいい感じに学習してくれるので，ないものはNULLのままおいておく

In [18]:
target_col = "gesture"

def feature_engineering(df,df_demo):
    df_copy = df.copy()
    df_demo_copy = df_demo.copy()

    df_copy = df_copy.drop(columns=["sequence_type","orientation","behavior","phase"])
    df_set = pd.merge(df_copy, df_demo_copy, on="subject", how="left")

    print("  Calculating engineered IMU features (magnitude, angle)...")
    df_set['acc_mag'] = np.sqrt(df_set['acc_x']**2 + df_set['acc_y']**2 + df_set['acc_z']**2)
    df_set['rot_angle'] = 2 * np.arccos(df_set['rot_w'].clip(-1, 1))

    df_set['acc_mag_jerk'] = df_set.groupby('sequence_id')['acc_mag'].diff().fillna(0)
    df_set['rot_angle_vel'] = df_set.groupby('sequence_id')['rot_angle'].diff().fillna(0)

    demographic_cols = [c for c in df_set.columns if c not in['subject']]

    imu_cols = [c for c in df_set.columns if c.startswith('acc_')]
    imu_cols.extend([c for c in df_set.columns if c.startswith('rot_') ])
    thm_cols = [c for c in df_set.columns if c.startswith('thm_') ]
    tof_cols = [c for c in df_set.columns if c.startswith('thof_') ]

    sensor_cols = imu_cols + thm_cols + tof_cols

    aggregations = {}
    for c in sensor_cols:
        aggregations[f"{c}_mean"] = (c, 'mean')
        aggregations[f"{c}_std"] = (c, 'std')
        aggregations[f"{c}_var"] = (c, 'var')
        aggregations[f"{c}_q25"] = (c, lambda x: x.quantile(0.25))
        aggregations[f"{c}_q50"] = (c, 'median')
        aggregations[f"{c}_q75"] = (c, lambda x: x.quantile(0.75))
        aggregations[f"{c}_max"] = (c, 'max')
        aggregations[f"{c}_min"] = (c, 'min')

    for c in demographic_cols + [target_col]:
        # 新しいカラム名と元のカラム名は同じ
        aggregations[c] = (c, 'first')

    df_group = df_set.groupby('sequence_id').agg(**aggregations)

    return df_group


In [32]:
train_df.head()

,acc_x_mean,acc_x_std,acc_x_var,acc_x_q25,acc_x_q50,acc_x_q75,acc_x_max,acc_x_min,acc_y_mean,acc_y_std,acc_y_var,acc_y_q25,acc_y_q50,acc_y_q75,acc_y_max,acc_y_min,acc_z_mean,acc_z_std,acc_z_var,acc_z_q25,acc_z_q50,acc_z_q75,acc_z_max,acc_z_min,acc_mag_mean,acc_mag_std,acc_mag_var,acc_mag_q25,acc_mag_q50,acc_mag_q75,acc_mag_max,acc_mag_min,acc_mag_jerk_mean,acc_mag_jerk_std,acc_mag_jerk_var,acc_mag_jerk_q25,acc_mag_jerk_q50,acc_mag_jerk_q75,acc_mag_jerk_max,acc_mag_jerk_min,rot_w_mean,rot_w_std,rot_w_var,rot_w_q25,rot_w_q50,rot_w_q75,rot_w_max,rot_w_min,rot_x_mean,rot_x_std,rot_x_var,rot_x_q25,rot_x_q50,rot_x_q75,rot_x_max,rot_x_min,rot_y_mean,rot_y_std,rot_y_var,rot_y_q25,rot_y_q50,rot_y_q75,rot_y_max,rot_y_min,rot_z_mean,rot_z_std,rot_z_var,rot_z_q25,rot_z_q50,rot_z_q75,rot_z_max,rot_z_min,rot_angle_mean,rot_angle_std,rot_angle_var,rot_angle_q25,rot_angle_q50,rot_angle_q75,rot_angle_max,rot_angle_min,rot_angle_vel_mean,rot_angle_vel_std,rot_angle_vel_var,rot_angle_vel_q25,rot_angle_vel_q50,rot_angle_vel_q75,rot_angle_vel_max,rot_angle_vel_min,thm_1_mean,thm_1_std,thm_1_var,thm_1_q25,thm_1_q50,thm_1_q75,thm_1_max,thm_1_min,thm_2_mean,thm_2_std,thm_2_var,thm_2_q25,thm_2_q50,thm_2_q75,thm_2_max,thm_2_min,thm_3_mean,thm_3_std,thm_3_var,thm_3_q25,thm_3_q50,thm_3_q75,thm_3_max,thm_3_min,thm_4_mean,thm_4_std,thm_4_var,thm_4_q25,thm_4_q50,thm_4_q75,thm_4_max,thm_4_min,thm_5_mean,thm_5_std,thm_5_var,thm_5_q25,thm_5_q50,thm_5_q75,thm_5_max,thm_5_min,row_id,sequence_id,sequence_counter,gesture,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,rot_z,thm_1,thm_2,thm_3,thm_4,thm_5,tof_1_v0,tof_1_v1,tof_1_v2,tof_1_v3,tof_1_v4,tof_1_v5,tof_1_v6,tof_1_v7,tof_1_v8,tof_1_v9,tof_1_v10,tof_1_v11,tof_1_v12,tof_1_v13,tof_1_v14,tof_1_v15,tof_1_v16,tof_1_v17,tof_1_v18,tof_1_v19,tof_1_v20,tof_1_v21,tof_1_v22,tof_1_v23,tof_1_v24,tof_1_v25,tof_1_v26,tof_1_v27,tof_1_v28,tof_1_v29,tof_1_v30,tof_1_v31,tof_1_v32,tof_1_v33,tof_1_v34,tof_1_v35,tof_1_v36,tof_1_v37,tof_1_v38,tof_1_v39,tof_1_v40,tof_1_v41,tof_1_v42,tof_1_v43,tof_1_v44,tof_1_v45,tof_1_v46,tof_1_v47,tof_1_v48,tof_1_v49,tof_1_v50,tof_1_v51,tof_1_v52,tof_1_v53,tof_1_v54,tof_1_v55,tof_1_v56,tof_1_v57,tof_1_v58,tof_1_v59,tof_1_v60,tof_1_v61,tof_1_v62,tof_1_v63,tof_2_v0,tof_2_v1,tof_2_v2,tof_2_v3,tof_2_v4,tof_2_v5,tof_2_v6,tof_2_v7,tof_2_v8,tof_2_v9,tof_2_v10,tof_2_v11,tof_2_v12,tof_2_v13,tof_2_v14,tof_2_v15,tof_2_v16,tof_2_v17,tof_2_v18,tof_2_v19,tof_2_v20,tof_2_v21,tof_2_v22,tof_2_v23,tof_2_v24,tof_2_v25,tof_2_v26,tof_2_v27,tof_2_v28,tof_2_v29,tof_2_v30,tof_2_v31,tof_2_v32,tof_2_v33,tof_2_v34,tof_2_v35,tof_2_v36,tof_2_v37,tof_2_v38,tof_2_v39,tof_2_v40,tof_2_v41,tof_2_v42,tof_2_v43,tof_2_v44,tof_2_v45,tof_2_v46,tof_2_v47,tof_2_v48,tof_2_v49,tof_2_v50,tof_2_v51,tof_2_v52,tof_2_v53,tof_2_v54,tof_2_v55,tof_2_v56,tof_2_v57,tof_2_v58,tof_2_v59,tof_2_v60,tof_2_v61,tof_2_v62,tof_2_v63,tof_3_v0,tof_3_v1,tof_3_v2,tof_3_v3,tof_3_v4,tof_3_v5,tof_3_v6,tof_3_v7,tof_3_v8,tof_3_v9,tof_3_v10,tof_3_v11,tof_3_v12,tof_3_v13,tof_3_v14,tof_3_v15,tof_3_v16,tof_3_v17,tof_3_v18,tof_3_v19,tof_3_v20,tof_3_v21,tof_3_v22,tof_3_v23,tof_3_v24,tof_3_v25,tof_3_v26,tof_3_v27,tof_3_v28,tof_3_v29,tof_3_v30,tof_3_v31,tof_3_v32,tof_3_v33,tof_3_v34,tof_3_v35,tof_3_v36,tof_3_v37,tof_3_v38,tof_3_v39,tof_3_v40,tof_3_v41,tof_3_v42,tof_3_v43,tof_3_v44,tof_3_v45,tof_3_v46,tof_3_v47,tof_3_v48,tof_3_v49,tof_3_v50,tof_3_v51,tof_3_v52,tof_3_v53,tof_3_v54,tof_3_v55,tof_3_v56,tof_3_v57,tof_3_v58,tof_3_v59,tof_3_v60,tof_3_v61,tof_3_v62,tof_3_v63,tof_4_v0,tof_4_v1,tof_4_v2,tof_4_v3,tof_4_v4,tof_4_v5,tof_4_v6,tof_4_v7,tof_4_v8,tof_4_v9,tof_4_v10,tof_4_v11,tof_4_v12,tof_4_v13,tof_4_v14,tof_4_v15,tof_4_v16,tof_4_v17,tof_4_v18,tof_4_v19,tof_4_v20,tof_4_v21,tof_4_v22,tof_4_v23,tof_4_v24,tof_4_v25,tof_4_v26,tof_4_v27,tof_4_v28,tof_4_v29,tof_4_v30,tof_4_v31,tof_4_v32,tof_4_v33,tof_4_v34,tof_4_v35,tof_4_v36,tof_4_v37,tof_4_v38,tof_4_v39,tof_4_v40,tof_4_v41,tof_4_v42,tof_4_v43,tof_4_v44,tof_4_v45,tof_4_v46,tof_4_v47,tof_4_v48,tof_4_v49,tof_4_v50,tof_4_v51,tof_4_v52,tof_4_v53

In [ ]:
train_df = feature_engineering(train,train_demo)

  Calculating engineered IMU features (magnitude, angle)...


In [33]:
le = LabelEncoder()
y = le.fit_transform(train_df[target_col])
X = train_df.drop(columns=[target_col, "sequence_id","row_id","sequence_counter"])

if MODE == 'KAGGLE':
    joblib.dump(le, 'le.joblib')
if MODE == 'LOCAL':
    file_path = EXPORT_DIR / 'le.joblib'
    joblib.dump(le, file_path)

In [34]:
def competition_metric(y_true, y_pred, le_instance, all_original_gestures):
    """
    Competition metric calculation
    """
    bfrb_gestures = [g for g in all_original_gestures if g in le_instance.classes_]
    
    # Binary F1: All are Target in this filtered dataset
    y_true_binary = np.ones_like(y_true, dtype=int)
    y_pred_binary = np.ones_like(y_pred, dtype=int)
    binary_f1 = f1_score(y_true_binary, y_pred_binary, average='binary', pos_label=1, zero_division=0)
    
    # Macro F1: specific gesture classification
    macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    
    final_score = (binary_f1 + macro_f1) / 2
    return final_score, binary_f1, macro_f1

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []
models = []

all_original_gestures_in_train = train_df['gesture'].unique()

callbacks = [lgb.early_stopping(stopping_rounds=100, verbose=100)]

# LightGBM model with cross-validation
print("\nTraining LightGBM models with cross-validation...")
for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    print(f"\nFold {fold + 1}/5")
    
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]
    
    # LightGBM model with GPU acceleration
    model = lgb.LGBMClassifier(
        objective='multiclass',
        n_estimators= 1000,
        learning_rate= 0.08,
        max_depth= 15,
        reg_alpha= 0.8,
        lambda_l2= 4.0,  
        num_leaves=31, 
        min_child_samples= 32,
        colsample_bytree= 0.85,
        subsample= 0.5,
        subsample_freq=0,
        cat_smooth=20.0,
        is_unbalance=True,
        max_bin=127,
        verbose=-1,  
        metric='multi_logloss',   
        device='gpu',  
    )
    
    # Train model with verbose output
    model.fit(
        X_train_fold, y_train_fold,
        eval_set=[(X_val_fold, y_val_fold)],  
        eval_metric='multi_logloss',  
        callbacks=callbacks
    )
    
    # Predict
    y_pred_fold = model.predict(X_val_fold)
    
    # Calculate score
    score, binary_f1, macro_f1 = competition_metric(
        y_val_fold, y_pred_fold, le, all_original_gestures_in_train
    )
    
    cv_scores.append(score)
    models.append(model)
    if MODE == 'KAGGLE':
        joblib.dump(le, 'le.joblib')
    if MODE == 'LOCAL':
        file_path = EXPORT_DIR / f'model_lgb{fold}.joblib'
        joblib.dump(model, file_path)
    
    print(f"Fold {fold + 1} - Competition Score: {score:.4f} (Binary F1: {binary_f1:.4f}, Macro F1: {macro_f1:.4f})")


print(f"\nCross-validation results:")
print(f"Mean CV Score: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores) * 2:.4f})")
print(f"Individual fold scores: {cv_scores}")

# Train final model on all data with GPU acceleration
print("\nTraining final model on all training data...")


Training LightGBM models with cross-validation...

Fold 1/5
Training until validation scores don't improve for 100 rounds
